In [1]:
import os
import gc
import sys
import time
import json 
import random
import logging 
import warnings
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', 500)
import scipy.io as sio
import geopandas as gpd
from ctypes import c_double
# geometry
import shapely.wkt
import shapely.ops
from shapely.wkt import loads
from shapely.ops import substring
from shapely.geometry import Point
import scipy.spatial.distance
# raster
import rasterio as rio
import rasterio.plot 
from rasterio.warp import transform
from osgeo import gdal
from PIL import Image
# plot
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

### dir
home_dir = '/home/bingyu/Documents/spatial_queue' # os.environ['HOME']+'/spatial_queue'
work_dir = '/home/bingyu/Documents/spatial_queue' # os.environ['WORK']+'/spatial_queue'
scratch_dir = '/home/bingyu/Documents/spatial_queue' # os.environ['OUTPUT_FOLDER']

### user
sys.path.insert(0, home_dir)
import util.haversine as haversine
from model.queue_class import Network, Node, Link, Agent
from game.extract_vehicle_locations import extract_vehicle_locations

import warnings
warnings.filterwarnings('error', message='Creating an ndarray from ragged*')

random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)

In [10]:
def preparation(random_seed=0, fire_id=None, comm_id=None, vphh=None, visitor_cnts=None, contra_id=None, link_closed_time=None, closed_mode=None, shelter_scen_id=None, scen_nm=None, trace_agent_list=None):
    ### logging and global variables

    project_location = '/projects/bolinas_civic'
    network_file_edges = project_location + '/network_inputs/bolinas_edges_sim.csv'
    network_file_nodes = project_location + '/network_inputs/bolinas_nodes_sim.csv'
    network_file_special_nodes = project_location + '/network_inputs/bolinas_special_nodes.json'
    demand_files = [project_location + '/demand_inputs/od_csv/resident_visitor_od_rs{}_vphh{}_visitor{}_comm{}.csv'.format(random_seed, vphh, visitor_cnts, comm_id)]
    simulation_outputs = '/simulation_outputs' ### scratch_folder

    scen_nm = scen_nm
    logging.basicConfig(filename=scratch_dir+project_location+simulation_outputs+'/log/{}.log'.format(scen_nm), filemode='w', format='%(asctime)s - %(message)s', level=logging.INFO, force=True)
    logging.info(scen_nm)
    print('log file created for {}'.format(scen_nm))

    ### network
    with open(work_dir + network_file_special_nodes) as special_nodes_file:
        special_nodes = json.load(special_nodes_file)
    network = Network()
    network.dataframe_to_network(project_location=project_location, network_file_edges = network_file_edges, network_file_nodes = network_file_nodes, special_nodes=special_nodes, scen_nm=scen_nm)
    network.add_connectivity()

    ### demand
    network.add_demand(demand_files = demand_files)
    logging.info('total numbers of agents taken {}'.format(len(network.agents.keys())))

    return {'network': network}, {'scen_nm': scen_nm, 'simulation_outputs': simulation_outputs, 'comm_id': comm_id, 'special_nodes': special_nodes}, {}

def link_model(t, network, link_closed_time=None, closed_mode=None):
    
    # run link model
    for link_id, link in network.links.items(): 
        link.run_link_model(t, agent_id_dict=network.agents)

    return network

def node_model(t, network, special_nodes=None):
    # only run node model for those with vehicles waiting
    node_ids_to_run = set([link.end_nid for link in network.links.values() if len(link.queue_vehicles)>0])

    # run node model
    for node_id in node_ids_to_run:
        node = network.nodes[node_id] 
        n_t_move, transfer_links= node.run_node_model(t, node_id_dict=network.nodes, link_id_dict=network.links, agent_id_dict=network.agents, node2link_dict=network.node2link_dict, special_nodes=special_nodes)
        
    return network

def one_step(t, data, config, update_data):

    network = data['network']
    
    scen_nm, simulation_outputs, comm_id, special_nodes = config['scen_nm'], config['simulation_outputs'], config['comm_id'], config['special_nodes']

    ### update link travel time before rerouting
    reroute_freq = 300
    
    ### reset link congested counter
    for link in network.links.values():
        link.congested = 0
        if (t%100==0):
            link.update_travel_time_by_queue_length(network.g)

    if t==0:
        for agent_id, agent in network.agents.items():
            network.agents[agent_id].departure_time = np.random.randint(1, 100)

    ### agent model
    t_agent_0 = time.time()
    stopped_agents_list = []
    for agent_id, agent in network.agents.items():
        ### first remove arrived vehicles
        if agent.status == 'arrive':
            network.agents_stopped[agent_id] = (agent.status, t, agent.agent_type)
            stopped_agents_list.append(agent_id)
            continue
        ### find congested vehicles: spent too long in a link
        current_link = network.links[network.node2link_dict[(agent.current_link_start_nid, agent.current_link_end_nid)]]
        if (current_link.link_type != 'v') and (agent.current_link_enter_time is not None) and (t-agent.current_link_enter_time>3600*0.5):
            current_link.congested += 1
            if (t-agent.current_link_enter_time>3600*3):
                agent.status = 'shelter_a1'
        ### agents need rerouting
        # initial route 
        if (t==0) or (t%reroute_freq==agent_id%reroute_freq):
            routing_status = agent.get_path(t, g=network.g)
            agent.find_next_link(node2link_dict=network.node2link_dict)
            if agent_id == 0: print(agent.route)
        agent.load_vehicle(t, node2link_dict=network.node2link_dict, link_id_dict=network.links)
        ### remove passively sheltered vehicles immediately, no need to wait for node model
        if agent.status in ['shelter_p', 'shelter_a1', 'shelter_park']:
            current_link.queue_vehicles = [v for v in current_link.queue_vehicles if v!=agent_id]
            current_link.run_vehicles = [v for v in current_link.run_vehicles if v!=agent_id]
            network.nodes[agent.current_link_end_nid].shelter_counts += 1
            network.agents_stopped[agent_id] = (agent.status, t, agent.agent_type)
            stopped_agents_list.append(agent_id)
    for agent_id in stopped_agents_list:
        del network.agents[agent_id]
    t_agent_1 = time.time()

    ### link model
    ### Each iteration in the link model is not time-consuming. So just keep using one process.
    t_link_0 = time.time()
    network = link_model(t, network)
    t_link_1 = time.time()
    
    ### node model
    t_node_0 = time.time()
    network = node_model(t, network, special_nodes=special_nodes)
    t_node_1 = time.time()
        
    # stop
    if len(network.agents)==0:
        logging.info("all agents arrive at destinations")
        return network, 'stop'
    else:
        return network, 'continue'

In [33]:
comm_id='1' # '1', '2', '3'
vphh=1 # vehicles per household
visitor_cnts=0

# reset random seed
random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)

# base network as the base layer of plotting
roads_df = pd.read_csv(home_dir + '/projects/bolinas_civic/network_inputs/bolinas_edges_sim.csv')
roads_gdf = gpd.GeoDataFrame(roads_df, crs='epsg:4326', geometry=roads_df['geometry'].map(loads)).to_crs(26910)

# set scenario name
scen_nm = "r{}_vphh{}_vistor{}_comm".format(random_seed, vphh, visitor_cnts, comm_id)

data, config, update_data = preparation(random_seed=random_seed, comm_id=comm_id, vphh=vphh, visitor_cnts=visitor_cnts, scen_nm=scen_nm)

link_speed_dict = dict()
link_vehicles_dict = dict()
link_maxspeed_array = np.array([link.length/(link.fft+0.00001) for link_id, link in data['network'].links.items() if link.link_type != 'v'])
link_length_array = np.array([(link.length+0.00001) for link_id, link in data['network'].links.items() if link.link_type != 'v'])
link_id_array = np.array([link_id for link_id, link in data['network'].links.items() if link.link_type != 'v'])
print(link_id_array[0:10])

for t in range(0, 5400):
    # run simulation for one step
    network, status = one_step(t, data, config, update_data)
    link_vehicles_array = np.array([(len(link.run_vehicles) + len(link.queue_vehicles)) for link_id, link in data['network'].links.items() if link.link_type != 'v'])
    link_density_array = link_vehicles_array * 8 / link_length_array
    link_speed_array = np.where(link_density_array>=1, 0, link_maxspeed_array*(1-link_density_array))
#     print(link_speed_array[0:10])
    link_speed_dict[t] = link_speed_array
    link_vehicles_dict[t] = link_vehicles_array
print(link_speed_dict[199][0:10])

log file created for r0_vphh1_vistor0_comm
# agents from file  595
[0 1 2 3 4 5 6 7 8 9]
{150: 141, 141: 43, 43: 149, 149: 146, 146: 151, 151: 142, 142: 170, 170: 164, 164: 162, 162: 180, 180: 189, 189: 125, 125: 19, 19: 240, 240: 62, 62: 112, 112: 207, 207: 219, 219: 117, 117: 122, 122: 20, 20: 22, 22: 183, 183: 248, 248: 245, 245: 193, 193: 223, 223: 194, 194: 200, 200: 8, 8: 5, 5: 4, 4: 84, 84: 187, 187: 155}
{162: 130, 130: 129, 129: 252, 252: 91, 91: 128, 128: 190, 190: 191, 191: 216, 216: 106, 106: 122, 122: 20, 20: 22, 22: 183, 183: 248, 248: 245, 245: 193, 193: 223, 223: 194, 194: 200, 200: 8, 8: 5, 5: 4, 4: 84, 84: 187, 187: 155}
{20: 22, 22: 183, 183: 248, 248: 197, 197: 245, 245: 193, 193: 223, 223: 194, 194: 200, 200: 8, 8: 5, 5: 4, 4: 84, 84: 187, 187: 155}
{248: 245, 245: 193, 193: 223, 223: 194, 194: 200, 200: 8, 8: 5, 5: 4, 4: 84, 84: 187, 187: 155}
{193: 223, 223: 194, 194: 200, 200: 8, 8: 5, 5: 4, 4: 84, 84: 187, 187: 155}
{193: 223, 223: 194, 194: 200, 200: 8, 8: 5, 

In [34]:
def get_route(g, origin, destin):
    sp = g.dijkstra(origin, destin)
    sp_dist = sp.distance(destin)
    if (sp_dist>1e8):
        sp.clear()
        print('no path found')
        return {}
    else:
        sp_route = sp.route(destin)
        route = {}
        for (start_nid, end_nid) in sp_route:
            route[start_nid] = end_nid
        sp.clear()
        return route

def get_travel_time(network, player_origin, player_destin):
    
    current_link = 'n{}_vl'.format(player_origin)
    current_link_distance = 0
    current_link_angle = 0
    
    for t_p in range(5400):
        if t_p > 0:
            current_link_distance += link_speed_dict[t_p][current_link] ### the link_speed_dict sequence may not be the same as the link_id sequence
#             print(t_p, current_link_distance)
        
        if (t_p == 0) or (current_link_distance >= network.links[current_link].length):
            
            if network.links[current_link].end_nid == player_destin:
                print('Reach destination in {}'.format(t_p))
                break
            
            # extra distance to next link
            current_link_distance = current_link_distance - network.links[current_link].length
            
            # reroute at intersection
            player_route_by_nodes = get_route(network.g, network.links[current_link].end_nid, player_destin)
            # print(player_route)
            
            ### move agent to to chosen link
            next_node = player_route_by_nodes[network.links[current_link].end_nid]
            current_link = network.node2link_dict[(network.links[current_link].end_nid, next_node)]
            print('new link {} at {}\n'.format(current_link, t_p))  
    
player_origin, player_destin = 143, 193
get_travel_time(network, player_origin, player_destin)

new link 419 at 0

new link 417 at 19

new link 279 at 37

new link 192 at 44

new link 190 at 65

new link 188 at 88

new link 186 at 109

new link 535 at 123

new link 537 at 180

new link 539 at 364

new link 541 at 413

new link 543 at 465

new link 600 at 991

new link 598 at 1082

new link 596 at 1126

new link 11 at 1278

new link 575 at 1295

Reach destination in 1865
